# Function Processing Module

In [2]:
import inspect
from inspect import signature
from inspect import isgeneratorfunction
from functools import wraps
from collections import Counter
from functools import partial

from pprint import pprint

import unittest
from pathlib import Path
from typing import List
import re
from sections import Rule, RuleSet
import text_reader as tp

## MapSig Class 
- An instance is a decorator used to wrap functions
- When creating an instance, a "Calling" signature is defined and the resulting wrapped function will appear to have that signature.
- The role of the decorator is to map the parameters in the signature of the wrapped function to the parameters signature for the original un-wrapped function.
- If parameters are present in the wrapped function signature that cannot be mapped to a parameter in the original un-wrapped function signature, those parameter values will be silently dropped.
- If required (positional) parameters are present in the original un-wrapped function signature that cannot be associated with a parameter (of any kind) in the wrapped function signature, an error will be raised when attempting to initialize the object.
- The decorator also assigns appropriate annotation to the wrapped function.
  - The wrapped function annotation specifies the new signature
  - The wrapped function annotation uses the name and doc string from the original un-wrapped function.



`context` parameter is unique.
- If function has `context` keyword parameter, then it can modify `context` content.
- If function does not have a `context` keyword parameter, but does have a `VAR_KEYWORD` parameter (`**kwargs`), then the `VAR_KEYWORD` parameter will receive the `context` content (but cannot modify it).

#### Allow processing functions to take specific keyword arguments from context

add a wrapper to trap unwanted context items

1. Get parameters and defaults for all arguments of the
   supplied function.
2. Generate a defaults dictionary for all keyword
   arguments.
3. When called, pass non-keyword arguments directly to
   the original function.
4. Update the defaults dictionary with calling
   keywords except the context keyword, if present.
5. If context is among the default keyword arguments
   and is supplied when called, add context to the keyword dictionary; otherwise search the supplied context for any keys in the defaults dictionary and use them to update the defaults dictionary.
10. If a **kwargs is present in the original function,
    pass all remaining context items using **context.



### Initialization
A MapSig object is defined by passing a dummy function that has teh desired signature for the final wrapped function.  e.g.

```python
def dummy_func(arg1: str, arg2='', **arg3)->str: pass

three_arg_wrapper = MapSig(dummy_func)

@three_arg_wrapper
def one_string_func(string1: str)->str:
    return string1.lower()

one_string_func('ABC', 'DEF', extra_arg='Nothing')
>>> 'abc'
```

### Mapping parameters between signatures.

The "Calling" signature is defined with sequence of argument definitions. 

**Argument Definition:**
- **Name** (str): The name of the argument.
- **Type** (str): _'Positional'_, _'Keyword'_, _'kwarg'_
  - All _'Positional'_ arguments must precede the first _'Keyword'_ argument.
  - A _'kwarg'_ argument is not required, but there can only be one _'kwarg'_ argument, and if present, it must be the last argument.
  - If the "Calling" signature includes a _'kwarg'_ argument and the un-wrapped function signature contains a parameter name matching the _'kwarg'_ argument, the _'kwarg'_ argument will be passed as a dictionary object to the the un-wrapped function.
  - If the "Calling" signature includes a _'kwarg'_ argument and the un-wrapped function signature **does not** contain a parameter name matching the _'kwarg'_ argument, but does contain it's own `**kwargs` type parameter, the _'kwarg'_ argument the _'kwarg'_ argument will be passed to the the un-wrapped function in the `**kwarg` style.
  - The _'kwarg'_ argument is expected to have the `Dict[str, Any]` data type so that it can be passed to the un-wrapped function in the `**kwarg` style if required.
- **Required** (bool, optional): If an argument is required, there must be a corresponding argument in the un-wrapped function signature.
  - Default is False.
  - Any _'Keyword'_ argument can be required regardless of order.
  - The "Calling" signature must begin with any required _'Positional'_ arguments. (A required _'Positional'_ argument may not follow any non-required _'Positional'_ arguments.)

#### "Calling" signature checks
- if Type is not one of [_'Positional'_, _'Keyword'_, _'kwarg'_], `ValueError` is raised.
- If a _'Positional'_ argument follows the first _'Keyword'_ argument, `ValueError` is raised.
- If a "required" _'Positional'_ argument follows a "non-required" _'Positional'_ argument, `ValueError` is raised.

### \_\_call__ method
- The `__call__(func)` method is used to wrap `func`.
- It compares the `func` signature with the defined "Calling" signature

1. _'Keyword'_ arguments are matched with `func` _'positional or keyword'_ and  _'keyword-only'_ parameters that have the same name.

2. If the `func` signature **does not** contain a 'variadic keyword' (`**kwargs`) parameter, all _'Keyword'_ arguments **following** the first matched _'Keyword'_ argument are not passed to `func`. If any of those arguments is 'Required' a `ValueError` is raised.

3. A sequence is created from the _'Positional'_ arguments and all arguments **preceding** the first matched _'Keyword'_ argument.
  
4. A second sequence is created from the _'positional-only'_ and _'positional or keyword'_ `func` parameters.

5. The two sequences are mapped 1-to-1 until one of the sequences is exhausted. 

6. If the second sequence (containing `func` parameters) is exhausted first, and the `func` signature **does not** contain a 'variadic positional' (`*args`) parameter, any remaining _'Positional'_ arguments are not passed to `func`.  If any of those arguments is 'Required' a `ValueError` is raised.

- defaults
- `*args`
- `**kwargs`
- The _'kwarg'_ argument


1. If the `func` signature **contains** a 'variadic keyword' (`**kwargs`), all un matched _'Keyword'_ arguments are passed to `func` via the `**kwargs` assignment.



  
> - Without a list of valid signatures it will assume that positional arguments 
>   come in the same order as in the final signature.
> - Additional keyword arguments can be extracted from the 'varkw' argument 
>   (if it exists) in the final signature.
> - Additional keyword arguments can be supplied during instance creation and passed
>   to the called function.

Move standard_action, sig_match, and set_method into the module

## Classes as function wrappers
- Class objects are decorators

- Action class
  - Convert *Action* strings into standard functions

- StandardSig class
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.

- FunctionProcessor Class
  - Apply specific Action and StandardSig instances
  - Add attribute indicating if the supplied function is a generator.

### StandardSig Class 
  - Check the function argument signature against expected possible signatures.
  - Wrap the function with a standard argument signature.
  - Assign annotation to resulting function based on final signature and wrapped function.

A sequence of variable definitions is given representing the desired final signature.

has __call__ method 



Definition
- Final (required) signature 
- Valid signatures
  
> - Without a list of valid signatures it will assume that positional arguments 
>   come in the same order as in the final signature.
> - Additional keyword arguments can be extracted from the 'varkw' argument 
>   (if it exists) in the final signature.
> - Additional keyword arguments can be supplied during instance creation and passed
>   to the called function.

##### Example:

**Required signature:**
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Name: str  |Type: [type, str]|Required: bool  |Positional: bool|Notes: str|
|-----------|-----------------|----------------|----------------|-----------|
|test_object|SourceItem       |True            |True            |Must be present, cannot be in context|
|event      |TriggerEvent     |False           |True            |can be in context?|
|context    |'varkw'          |False           |True            |context  or **context|


**Valid signatures**

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):


|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|


**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

```python
rule_sig = {
    (1, False): lambda func, item, event, context: func(item),
    (1, True): lambda func, item, event, context: func(item, **context),
    (2, False): lambda func, item, event, context: func(item, event),
    (2, True): lambda func, item, event, context: func(item, event, **context),
    (3, False): lambda func, item, event, context: func(item, event, context)
    }
```

### FunctionProcessor Class

- Class objects are decorators / functions that 
  1. Convert *Action* strings into standard functions
  3. Check the function argument signature against expected possible signatures.
  4. Wrap the function with a standard argument signature.
  5. Test whether a supplied function is a generator.
  


- Take text processing functions and convert them into *actions*

- Convert set_method, sig_match, standard_action into a helper class

The class can be initialized with signature and action definitions to create different objects for different function groups. When the object is applied to a function or string, it returns a corresponding function with the expected argument signature.

The class object is a custom decorator function.

https://stackoverflow.com/questions/76304643/reuse-of-function-signature-type-hints-and-intellisense-args-kwargs


### Standard Actions
*Convert a Method name to a Standard Function.*

Take the name of a standard actions and return the matching function.

**Arguments:**
- action_name (str): The name of a standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises: **
- ValueError If the action_name or method_type supplied are not one of
    the valid action names or method types.

**Returns:**
- (ProcessFunc, RuleFunc): One of the standard action functions.

### sig_match
*Convert the supplied function with a standard signature.*

The conversion is based on number of arguments without defaults and the
presence of a var keyword argument (**kwargs):

**Arguments:**
- given_method (MethodOptions): A function to be used as a rule
        method or a process method.
- sig_type (str, optional): The type of argument signature desired.  Can
        be one of: 'Process' or 'Rule'.  Defaults to 'Process'.

**Raises:**
- ValueError If given_method does not have one of the expected
    argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature


### set_method
*Convert the supplied function or action name to a Function with
    the standard signature.*

See the standard_action function for valid action names. See the sig_match
function for valid function argument signatures.
Add a special attribute to the returned function indicating if it is a
generator function.

**Arguments:**
- given_method (RuleMethodOptions): A function, or the name of a
    standard action.
- method_type (str): The type of action function expected.
    One of ['Process', 'Rule']. Defaults to 'Process'

**Raises:**
- ValueError If rule_method is a string and is not one of the
    valid action names, or if rule_method is a function and does not
    have one of the valid argument signature types.

**Returns:**
- (ProcessFunc, RuleFunc): A function with the standard Rule or Process
        Method argument signature

### Method Types

#### rule
- rule_method(item, context) is not allowed; use:
    - rule_method(item, **context)          or
    - rule_method(item, event, context)     instead.

*Required signature:*
> `rule_method(test_object: SourceItem, event: TriggerEvent, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, event)                         |2   |None   |                   |
|func(item, event [, other(s)=defaults(s)])|1   |None   |                   |
|func(item, event, **context)              |2   |context|                   |
|func(item, event, context)                |3   |None   |Expected           |
|func(item, event, [other(s),] **context)  |3+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.
- 'Value': return the self.event.test_value object.
- 'Name': return the self.event.test_name object.

#### process
- Also applies to Sentinel and Assemble methods.

*Required signature:*
> `process_method(test_object: SourceItem, context)`

|Signature                                 |Args|varkw  |Notes              |
|------------------------------------------|----|-------|-------------------|
|func(item)                                |1   |None   |                   |
|func(item  [, other(s)=defaults(s)])      |1   |None   |                   |
|func(item, **context)                     |1   |context|                   |
|func(item, context)                       |2   |None   |Expected           |
|func(item  [, other(s)] **context)        |2+  |context|Not Yet Implemented|

**Valid Action Names**
- 'Original': return the original item supplied.
- 'Blank': return ''  (an empty string).
- 'None': return None.



#### Function Signature options

In [6]:
[k.description for k in inspect._ParameterKind.__members__.values()]

['positional-only',
 'positional or keyword',
 'variadic positional',
 'keyword-only',
 'variadic keyword']

In [10]:
[k.name for k in inspect._ParameterKind.__members__.values()]

['POSITIONAL_ONLY',
 'POSITIONAL_OR_KEYWORD',
 'VAR_POSITIONAL',
 'KEYWORD_ONLY',
 'VAR_KEYWORD']

**kind**
> Describes how argument values are bound to the parameter. The possible values are accessible via Parameter (like Parameter.KEYWORD_ONLY), and support comparison and ordering, in the following order:

|Name|Description|Meaning|
|----|-----------|-------|
POSITIONAL_ONLY|`'positional-only'`|Value must be supplied as a positional argument. Positional only parameters are those which appear before a / entry (if present) in a Python function definition.|
|POSITIONAL_OR_KEYWORD|`'positional or keyword'`|Value may be supplied as either a keyword or positional argument (this is the standard binding behaviour for functions implemented in Python.)|
|VAR_POSITIONAL|`'variadic positional'`|A tuple of positional arguments that aren’t bound to any other parameter. This corresponds to a *args parameter in a Python function definition.|
|KEYWORD_ONLY|`'keyword-only'`|Value must be supplied as a keyword argument. Keyword only parameters are those which appear after a * or *args entry in a Python function definition.|
|VAR_KEYWORD|`'variadic keyword'`|A dict of keyword arguments that aren’t bound to any other parameter. This corresponds to a **kwargs parameter in a Python function definition.|


**Example:** 

In [7]:
def foo(a, /, b, *c, d, e=10, **f):
    pass

sig = signature(foo)


*Count the number of parameters of each type*

In [8]:
Counter(param.kind.description for param in sig.parameters.values())


Counter({'keyword-only': 2,
         'positional-only': 1,
         'positional or keyword': 1,
         'variadic positional': 1,
         'variadic keyword': 1})

In [9]:
Counter(param.kind.name for param in sig.parameters.values())


Counter({'KEYWORD_ONLY': 2,
         'POSITIONAL_ONLY': 1,
         'POSITIONAL_OR_KEYWORD': 1,
         'VAR_POSITIONAL': 1,
         'VAR_KEYWORD': 1})

*List all parameters, their type and their default (if it exists):*

In [65]:
for param in sig.parameters.values():
    if param.default is param.empty:
        dft = ''
    else:
        dft = param.default
    print(f'{param.name}\t{param.kind.description:25s}{dft}')


a	positional-only          
b	positional or keyword    
c	variadic positional      
d	keyword-only             
e	keyword-only             10
f	variadic keyword         


*print all keyword-only arguments without default values:*

In [60]:
for param in sig.parameters.values():
    if (param.kind == param.KEYWORD_ONLY and
                       param.default is param.empty):
        print('Parameter:', param)


Parameter: d


*Calling signature:*
> `rule_method(test_object: Any, event: TriggerEvent, context)`

*Allowed signatures:*
> `func(arg1, event, context)`  *Both event and context as keyword args*<br>
> `func(arg1, arg2,   arg3  )`  *Final signature*<br>
> `func(arg1, arg2, **arg3  )`  *Immutable context*<br>
> `func(arg1, arg2          )`  *No context*<br>
> `func(arg1,       **arg3  )`  *No event*<br>
> `func(arg1                )`
> `func(arg1, event         )`  *event as keyword arg*<br>
> `func(arg1, context       )`  *context as keyword arg*<br>
> `func(arg1, context, event)`  *Reversed event and context keyword args*<br>

> `func(*args, **kwargs)`  *Generic*<br>

> `func(arg1, other_arg=`\<Any> ...`          )`  *Arguments with defaults*<br>
> `func(arg1, other_arg=`\<Any> ...`, **arg3  )`  *other_arg is taken from context (if present)*<br>



1. 'VAR_POSITIONAL' parameter present
   1. Number of parameters before 'VAR_POSITIONAL' parameter less than or equal to expected number.
   2. All parameter names after the 'VAR_POSITIONAL' parameter match.
   3. Required 'KEYWORD_ONLY' parameters present

2. 'VAR_KEYWORD' parameter present
   1. Number of positional parameters before 'VAR_KEYWORD' parameter less than or equal to expected number.
   2. 'context' is not the name of any parameter except 'VAR_KEYWORD'
   3. Required 'POSITIONAL_ONLY' parameters present

3. Number of parameters less than or equal to expected number.
   1. No unmatched parameter names present (order irrelevant)
      1. All parameter names present match.
      2. All required parameters present. 
   2. Unmatched parameter names present.  (order of matched parameter names irrelevant)
      1. All unmatched parameter names precede the matched parameter names.
      2. All unmatched parameter do not map by order to a matched parameter name. 
      3. Required 'KEYWORD_ONLY' parameters present.


4. Unmatched keyword parameter names present.
   1. All unmatched positional parameter names precede the matched parameter names.
   2. All unmatched parameters do not map by order to a matched parameter name. 
   3. Required 'KEYWORD_ONLY' parameters present.



5. Number of parameters greater than expected number.
6. All parameter names present match (but some non-required parameters missing, order still irrelevant)

- 'POSITIONAL_ONLY'
- 'POSITIONAL_OR_KEYWORD'
- 'VAR_POSITIONAL'
- 'KEYWORD_ONLY'
- 'VAR_KEYWORD'

In [11]:
def bar(a=1, b=2, c=3):
    print(a)
    print(b)
    print(c)

bar(b=5, c=6, a=4)

4
5
6


In [15]:
bar(5, c=7, b=6)

5
6
7


#### Testing supplied function signature
1. Check for matching argument names
2. check for VAR_KEYWORD

In [17]:
def parse_prescribed_dose(line, event, **context)->List[List[str]]:# pylint: disable=unused-argument
    '''Split "Prescribed dose [cGy]" into 2 lines:
        Prescribed dose
        Prescribed dose Unit
        '''
    parse_template = [
        ['Prescribed dose', '{dose}'],
        ['Prescribed dose Unit', '{unit}']
        ]
    match_results = event.test_value.groupdict()
    if match_results['dose'] == 'not defined':
        parsed_lines = [
            ['Prescribed dose', ''],
            ['Prescribed dose Unit', '']
            ]
    else:
        parsed_lines = [
            [string_item.format(**match_results) for string_item in line_tmpl]
            for line_tmpl in parse_template
            ]
    for line in parsed_lines:
        yield line


In [16]:
#%% Test Text
test_lines = '''
Prescribed dose [cGy]: 5000.0
Prescribed dose [cGy]: not defined

Plan Status: Unapproved
Plan Status: Treatment Approved Thursday, January 02, 2020 12:55:56 by gsal

% for dose(%): 100.0
% for dose (%): not defined
'''


In [18]:
inspect.isfunction(parse_prescribed_dose)

True

In [19]:
a = signature(parse_prescribed_dose)


In [22]:
str(a)

'(line, event, **context) -> List[List[str]]'

In [21]:
len(a.parameters)

3

In [34]:
b = [param.kind for param in a.parameters.values()]
b[0]

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [38]:
inspect._ParameterKind.__members__

mappingproxy({'POSITIONAL_ONLY': <_ParameterKind.POSITIONAL_ONLY: 0>,
              'POSITIONAL_OR_KEYWORD': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>,
              'VAR_POSITIONAL': <_ParameterKind.VAR_POSITIONAL: 2>,
              'KEYWORD_ONLY': <_ParameterKind.KEYWORD_ONLY: 3>,
              'VAR_KEYWORD': <_ParameterKind.VAR_KEYWORD: 4>})

In [31]:



Counter(param.kind.description for param in a.parameters.values())

Counter({'positional or keyword': 2, 'variadic keyword': 1})

In [ ]:
rule_sig = {
    (1, False): lambda func, item, event, context: func(item),
    (1, True): lambda func, item, event, context: func(item, **context),
    (2, False): lambda func, item, event, context: func(item, event),
    (2, True):
        lambda func, item, event, context: func(item, event, **context),
    (3, False):
        lambda func, item, event, context: func(item, event, context)
    }
process_sig = {
    (1, False): lambda func, item, context: func(item),
    (2, False): lambda func, item, context: func(item, context),
    (1, True): lambda func, item, context: func(item, **context)
    }


In [ ]:

arg_spec = inspect.getfullargspec(given_method)
# Determine arg_count
if not arg_spec.args:
    if arg_spec.varargs:
        arg_count = 1
    else:
        arg_count = 0
elif not arg_spec.defaults:
    arg_count = len(arg_spec.args)
else:
    arg_count = len(arg_spec.args) - len(arg_spec.defaults)
    if arg_count == 0:
        arg_count = 1

# Determine presence of keyword argument catcher
has_varkw = arg_spec.varkw is not None


In [ ]:
list(a.parameters.values())[1].name

'event'

In [ ]:
b = a.bind_partial('l')
b

<BoundArguments (line='l')>

In [ ]:
b.apply_defaults()
b

<BoundArguments (line='l', context={})>

In [ ]:
b.signature

<Signature (line, event, **context) -> List[List[str]]>

In [ ]:
def date_parse(line, event)->List[List[str]]:
    '''If Date,don't split beyond first :'''
    parsed_line = [event.test_value, line.split(':',maxsplit=1)[1]]
    return parsed_line


In [ ]:
def approved_status_parse(line, event)->List[List[str]]:
    '''If Treatment Approved, Split "Plan Status" into 3 lines:
        Plan Status
        Approved on
        Approved by
        '''
    idx1 = line.find(event.test_value)
    idx2 = idx1 + len(event.test_value)
    idx3 = line.find('by')
    idx4 = idx3 + 3
    parsed_lines = [
        ['Plan Status', line[idx1:idx2]],
        ['Approved on', line[idx2:idx3]],
        ['Approved by', line[idx4:]]
        ]
    for line in parsed_lines:
        yield line

In [ ]:
def make_float(item, context):
    item_c = item.replace('Line','')
    try:
        num = float(item_c.strip())
    except:
        return None
    output =  f'Line {num:5.2f}'
    context['num'] = num
    return output

In [ ]:

if sig_type == 'Process':
    sig_function = process_sig.get((arg_count, has_varkw))
else:
    sig_function = rule_sig.get((arg_count, has_varkw))
if not sig_function:
    raise ValueError('Invalid function type.')
use_function = partial(sig_function, given_method)
func_name = getattr(given_method, '__name__', None)
if not func_name:
    if isinstance(given_method, partial):
        func_name = getattr(given_method.func, '__name__',
                            'PartialFunction')
    else:
        func_name = 'PartialFunction'
use_function.__name__ = func_name
return use_function


In [ ]:
class TestRuleExceptions(unittest.TestCase):

    @unittest.skip('Currently rule_method(item, context) is allowed')
    def test_no_event_arg(self):
        test_func = lambda item, context: str(item) + repr(context)
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_no_arg(self):
        def test_func():
            return "T"
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_many_arg(self):
        test_func = lambda item1, item2, event, context: 'a'
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method=test_func)

    def test_bad_action(self):
        with self.assertRaises(ValueError):
            invalid_rule = Rule('T', pass_method='Not an Action')

class TestRuleActions(unittest.TestCase):
    def test_original_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Original')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result, test_text)

    def test_event_action(self):
        test_text = 'Test Text'
        sentinel = 'Text'
        test_rule = Rule(sentinel, pass_method='Event')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertEqual(result.test_value, sentinel)

    def test_none_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='None')
        output = iter(test_rule(test_text))
        result = next(output)
        self.assertIsNone(result)

    def test_blank_action(self):
        test_text = 'Test Text'
        test_rule = Rule('Text', pass_method='Blank')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text)
        self.assertEqual(result, '')

class TestRuleFail(unittest.TestCase):
    def test_fail_method(self):
        test_text = 'Test Line'
        test_rule = Rule('Text', pass_method='Blank',
                            fail_method='Original')
        # Note this works because the Rule methods are not generator functions.
        result = test_rule.apply(test_text, {})
        self.assertEqual(result, test_text)
        result2 = test_rule.apply('Test Text', {})
        self.assertEqual(result2, '')
